# TorQ: Delete Everything


In [1]:
import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *

from env import *
from config import *

# Selective Deletion or All
DELETE_ALL=True

DELETE_CLUSTERS=True
DELETE_SCALING_GROUP=False
DELETE_DATAVIEW=False
DELETE_VOLUME=False
DELETE_DATABASE=False

if DELETE_ALL:
    DELETE_CLUSTERS=True
    DELETE_SCALING_GROUP=True
    DELETE_DATAVIEW=True
    DELETE_VOLUME=True
    DELETE_DATABASE=True


In [2]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = session.client(service_name='finspace')

## Delete all resources

In [3]:
# Clusters
#-------------------------------------------------------------------------------------------
if DELETE_CLUSTERS:
    print("Deleting Clusters...")

    delete_status = ['RUNNING', 'CREATING', 'ACTIVE']

    cdf = get_clusters(client, environmentId=ENV_ID)

    cluster_names = set([d['name'] for d in clusters if 'name' in d])

    # Clusters
    for c in cluster_names:
        this_c = get_kx_cluster(client=client, clusterName=c)
        if this_c is None:
            print(f"Not Found: {c}")
            continue
        elif this_c['status'] not in delete_status:
            print(f"Cannot Delete: {c} Status: {this_c['status']}")
            continue

        print(f"\tDelete: {c}")
        client.delete_kx_cluster(environmentId=ENV_ID, clusterName=c)
        print(f"\tDeleted: {c}")

    print(f"** Deleted {len(cluster_names)} Clusters **")

# Dataview
#-------------------------------------------------------------------------------------------
if DELETE_DATAVIEW:
    print()
    print(f"Deleting Dataview: {DBVIEW_NAME}...")
    this_view = get_kx_dataview(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)

    if this_view is None:
        print(f"Not Found: {DBVIEW_NAME}")
    elif this_view['status'] not in delete_status:
        print(f"Cannot Delete: {DBVIEW_NAME} Status: {this_view['status']}")
    else:
        print(f"delete: {DBVIEW_NAME}")
        client.delete_kx_dataview(environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)
        wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, status='DELETED', show_wait=True)
        print("** Deleted View **")

# Volume
#-------------------------------------------------------------------------------------------
if DELETE_VOLUME:
    print()
    print(f"Deleting Volume: {VOLUME_NAME}...")
    this_volume = get_kx_volume(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME)

    if this_volume is None:
        print(f"Not Found: {VOLUME_NAME}")
    elif this_volume['status'] not in delete_status:
        print(f"Cannot Delete: {VOLUME_NAME} Status: {this_volume['status']}")
    else:
        print(f"Delete: {VOLUME_NAME}")
        client.delete_kx_volume(environmentId=ENV_ID, volumeName=VOLUME_NAME)
        wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, status='DELETED', show_wait=True)
        print("** Deleted Volume **")

# Scaling Group
#-------------------------------------------------------------------------------------------
if DELETE_SCALING_GROUP:
    print()
    print(f"Deleting Scaling Group: {SCALING_GROUP_NAME}...")
    this_sg = get_kx_scaling_group(client=client, environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)

    if this_sg is None:
        print(f"Not Found: {SCALING_GROUP_NAME}")
    elif this_sg['status'] not in delete_status:
        print(f"Cannot Delete: {SCALING_GROUP_NAME} Status: {this_sg['status']}")
    else:
        print(f"Delete: {SCALING_GROUP_NAME}")
        client.delete_kx_scaling_group(environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)
        wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME, status='DELETED', show_wait=True)
        print("** Deleted Scaling Group **")

# Database
#-------------------------------------------------------------------------------------------
if DELETE_DATABASE:
    print()
    print(f"Deleting Database: {DB_NAME}...")
    this_db = get_kx_database(client=client, environmentId=ENV_ID, databaseName=DB_NAME)

    if this_db is None:
        print(f"Not Found: {DB_NAME}")
    else:
        print(f"delete: {DB_NAME}")
        client.delete_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)    
        print("** Deleted Database **")

print("\nDone Deleting")

Deleting Clusters...
	Delete: rdb1
	Deleted: rdb1
	Delete: gateway1
	Deleted: gateway1
	Delete: hdb1
	Deleted: hdb1
	Delete: feed1
	Deleted: feed1
	Delete: discovery1
	Deleted: discovery1
	Delete: monitor1
	Deleted: monitor1
** Deleted 6 Clusters **

Deleting Dataview: finspace-database_DBVIEW...
delete: finspace-database_DBVIEW
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:00:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:00:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:01:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:01:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:02:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:02:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:03:00, waiting 30 sec ...
Dataview: f

In [4]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-12-03 22:28:01.988693
